If using colab, install necessary libraries

In [4]:
! pip install python-lxml

ERROR: Could not find a version that satisfies the requirement python-lxml (from versions: none)
ERROR: No matching distribution found for python-lxml


In [9]:
import pandas as pd # 0.23.0
import numpy as np
import csv

In [37]:
def get_delimiter(file_path: str) -> str:
    with open(file_path, 'r') as csvfile:
        delimiter = str(csv.Sniffer().sniff(csvfile.read()).delimiter)
        return delimiter

In [69]:
fileName = 'combinations.csv'
df = pd.read_csv(fileName, sep = get_delimiter(fileName), on_bad_lines='skip')
#df['#FuelCode3'].unique()
#df.to_csv('combinations_cleaned.csv')
#df = df.replace(np.nan, '-')

In [70]:
tmpNull = 12345
df.iloc[:, 5:10] = df.iloc[:, 5:10].fillna(tmpNull).astype(int).replace(tmpNull, '-')
df = df.replace(np.nan, '-')
print("Are there any NaN values: ", df.isna().any().any())


Are there any NaN values:  False


In [102]:
df.iloc[:, 5:10] = df.iloc[:, 5:10].astype(str)
df

,Template,#VIN,#VehicleCategoryCode,ClassOfHybridVehicleCode,#FuelTypeCode,#FuelCode1,#FuelCode2,#FuelCode3,FuelCode4,Info manueller Change in DB: vehicleNatureCode,Bemerkung
0,C:\TESTDATENGENERATOR\Temp_NoHybrid_Bifuel.xml,BFE230703TEST0001,N1,Empty,B,10,40,-,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
1,C:\TESTDATENGENERATOR\Temp_OVC_Bifuel.xml,BFE230703TEST0002,N1,OVC-HEV,B,10,40,0,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
2,C:\TESTDATENGENERATOR\Temp_NOVC_Bifuel.xml,BFE230703TEST0003,N1,NOVC-HEV,B,10,40,0,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
3,C:\TESTDATENGENERATOR\Temp_NoHybrid_Bifuel.xml,BFE230703TEST0004,N1,Empty,B,10,40,-,-,38,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
4,C:\TESTDATENGENERATOR\Temp_OVC_Bifuel.xml,BFE230703TEST0005,N1,OVC-HEV,B,10,40,0,-,38,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
...,...,...,...,...,...,...,...,...,...,...,...
58,C:\TESTDATENGENERATOR\Temp_NOVC_Monofuel.xml,BFE230703TEST0059,N1,NOVC-HEV,M,20,-,0,-,-,"Mono-Fuel, Diesel/ elektrisch, tstof F"
59,C:\TESTDATENGENERATOR\Temp_OVC_Monofuel.xml,BFE230703TEST0060,N1,OVC-HEV,M,20,-,0,-,38,"Mono-Fuel, Diesel/ elektrisch, tstof F"
60,C:\TESTDATENGENERATOR\Temp_NOVC_Monofuel.xml,BFE230703TEST0061,N1,NOVC-HEV,M,20,-,0,-,38,"Mono-Fuel, Diesel/ elektrisch, tstof F"
61,C:\TESTDATENGENERATOR\Temp_OVC_Monofuel.xml,BFE230703TEST0062,M1,OVC-HEV,M,20,-,0,-,-,"Mono-Fuel, Diesel/ elektrisch, tstof F"


In [75]:
df['#FuelCode3'].unique()

array(['-', 0, 44], dtype=object)

---------------------------------------------------------------------------

Functions

In [49]:
dict_VCC = {'N1': '01', 'M1': '30'} # 35 and 38 need to be mapped
dict_CHVC = {'Empty': 'empty', 'OVC-HEV': 'vchev',
             'NOVC-HEV': 'nchev', 'OVC-FCHV': 'cfchv',
             'NOVC-FCHV': 'nfchv'}

In [81]:
tmp = df.iloc[0,:].copy().to_frame().T
tmp

,Template,#VIN,#VehicleCategoryCode,ClassOfHybridVehicleCode,#FuelTypeCode,#FuelCode1,#FuelCode2,#FuelCode3,FuelCode4,Info manueller Change in DB: vehicleNatureCode,Bemerkung
0,C:\TESTDATENGENERATOR\Temp_NoHybrid_Bifuel.xml,BFE230703TEST0001,N1,Empty,B,10,40,-,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"


In [114]:
def createVIN(row):
  newVIN = 'BFE' + dict_VCC[row['#VehicleCategoryCode'].values[0]] + dict_CHVC[row['ClassOfHybridVehicleCode'].values[0]] + str(row['#FuelTypeCode'].values[0])

  for col in ['#FuelCode1','#FuelCode2', '#FuelCode3']:

    val = str(row[col].values[0])
    print(val)

    if len(val) < 2:
      if val == '-':
        val = '$$'
      else:
        val = val + '$'

    newVIN = newVIN + val



  print(type(newVIN))

  return newVIN

In [115]:
createVIN(tmp)

10
40
-
<class 'str'>


'BFE01emptyB1040$$'

In [113]:
tmp

,Template,#VIN,#VehicleCategoryCode,ClassOfHybridVehicleCode,#FuelTypeCode,#FuelCode1,#FuelCode2,#FuelCode3,FuelCode4,Info manueller Change in DB: vehicleNatureCode,Bemerkung
0,C:\TESTDATENGENERATOR\Temp_NoHybrid_Bifuel.xml,BFE230703TEST0001,N1,Empty,B,10,40,-,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
